In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd  
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import random
import csv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
import re
import pickle



import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))


import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/zomato/zomato.csv", encoding='utf8')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df['rate'].value_counts()

NEW       2208
3.9/5     2098
3.8/5     2022
3.7/5     2011
3.9 /5    1874
          ... 
2.2 /5       7
2.0 /5       7
2.0/5        4
1.8 /5       3
1.8/5        2
Name: rate, Length: 64, dtype: int64

In [6]:
df = df[df['rate'] != 'NEW']
df = df[df['rate'] != np.nan]
df = df[df['rate'].notna()]
df = df[df['rate'] != '-']
df = df[df['reviews_list'] != ' ']

df.reset_index(inplace=True)

In [7]:
ratings=[]
rating=0
rating_int=0

for i in range(int(len(df['rate']))):

  rating = float(df['rate'][i][0:3])
  rating_int = int(rating)

  if (rating-rating_int > 0.5):
    rating = int(rating)+1
    ratings.append(rating)
  else:
    ratings.append(rating_int)

In [8]:
df['rating'] = ratings

df = df.drop(labels=['url', 'address', 'name', 'online_order', 'book_table', 'votes','phone', 'location', 'rest_type', 'cuisines', 'approx_cost(for two people)', 'listed_in(type)', 'listed_in(city)'], axis=1)
df_review = df.drop(labels=['rate', 'dish_liked', 'menu_item'], axis=1) 
df_dish = df.drop(labels=['rate','reviews_list', 'menu_item'], axis=1)

In [9]:
reviews = df_review['reviews_list']
ratings = df_review['rating']

In [10]:
length=[]
for i in reviews:
    length.append(len(i))
print("Mean Length of review: ", sum(length)/len(length))
print("Longest review: ", max(length), "\nShortest Review", min(length))

Mean Length of review:  11720.977631105245
Longest review:  1284117 
Shortest Review 2


In [11]:
dataset = {'reviews': reviews, 'ratings': ratings}
dataset = pd.DataFrame(dataset)

In [12]:
dataset.groupby(['ratings']).count()

,reviews
ratings,
2,288
3,13996
4,26793
5,588


In [13]:
dataset['ratings'].value_counts()

4    26793
3    13996
5      588
2      288
Name: ratings, dtype: int64

In [14]:
dataset.columns

Index(['reviews', 'ratings'], dtype='object')

In [15]:
x_train, x_test, y_train, y_test = train_test_split(dataset['reviews'], dataset['ratings'], test_size=0.2)
print(x_train.shape)
print(y_train.shape)

(33332,)
(33332,)


In [16]:
np.unique(y_train)

array([2, 3, 4, 5])

In [17]:
for i in range(len(dataset['reviews'])):
    review = dataset['reviews'][i]
    word_tokens = word_tokenize(review)
    new_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    dataset['reviews'][i] = new_sentence

dataset.to_csv('new_reviews.csv')